In [38]:
import numpy as np
import pandas as pd
import os
import dotenv
import psycopg
from sqlalchemy import create_engine

import dash
from dash import dcc, html
from dash.dependencies import Input, Output

import plotly.express as px
import plotly.figure_factory as ff



In [39]:

dotenv.load_dotenv()
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")

dbms ='postgresql'
package ='psycopg'
user ='postgres'
password = POSTGRES_PASSWORD
host ='localhost'
port = '5432'
db = 'contrans'

engine = create_engine(f'{dbms}+{package}://{user}:{password}@{host}:{port}/{db}')

In [40]:
# Create features that we need, but won't change depending on what the user does

myquery = '''
SELECT *
FROM members
'''

data = pd.read_sql(myquery, engine)

In [41]:
data

,bioguide_id,full_name,birthyear,image,office_address,phone,website,fec_id,bioname,chamber,left_right_ideology,state_abbrev,district_code,icpsr,party
0,A000055,Robert B. Aderholt,1965.0,https://www.congress.gov/img/member/a000055_20...,"272 Cannon House Office Building, Washington, ...",(202) 225-4876,https://aderholt.house.gov/,H6AL04098,"ADERHOLT, Robert",House,0.405,AL,4,29701,Republican
1,A000148,Jake Auchincloss,1988.0,https://www.congress.gov/img/member/67817e391f...,"1524 Longworth House Office Building, Washingt...",(202) 225-5931,https://auchincloss.house.gov,H0MA04192,"AUCHINCLOSS, Jake",House,-0.288,MA,4,22100,Democrat
2,A000369,Mark E. Amodei,1958.0,https://www.congress.gov/img/member/a000369_20...,"104 Cannon House Office Building, Washington, ...",(202) 225-6155,https://amodei.house.gov,H2NV02395,"AMODEI, Mark E.",House,0.384,NV,2,21196,Republican
3,A000370,Alma S. Adams,1946.0,https://www.congress.gov/img/member/a000370_20...,"2436 Rayburn House Office Building, Washington...",(202) 225-1510,https://adams.house.gov,H4NC12100,"ADAMS, Alma",House,-0.462,NC,12,21545,Democrat
4,A000371,Pete Aguilar,1979.0,https://www.congress.gov/img/member/a000371_20...,"108 Cannon House Office Building, Washington, ...",(202) 225-3201,https://aguilar.house.gov/,H2CA31125,"AGUILAR, Peter Rey",House,-0.324,CA,33,21506,Democrat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,W000830,George Whitesides,1974.0,https://www.congress.gov/img/member/68dc43db19...,"1504 Longworth House Office Building, Washingt...",(202) 225-1956,https://whitesides.house.gov/,H4CA27111,"WHITESIDES, George",House,-0.189,CA,27,22559,Democrat
541,W000831,James R. Walkinshaw,1982.0,https://www.congress.gov/img/member/68c1bd4ca9...,"2265 Rayburn House Office Building, Washington...",(202) 225-1492,https://walkinshaw.house.gov/,H6VA11066,"WALKINSHAW, James R.",House,-0.503,VA,11,22564,Democrat
542,Y000064,Todd Young,1972.0,https://www.congress.gov/img/member/y000064_20...,185 Dirksen Senate Office Building Washington...,(202) 224-5623,https://www.young.senate.gov,S0IN00194,"YOUNG, Todd",Senate,0.438,IN,0,21133,Republican
543,Y000067,Rudy Yakym III,1984.0,https://www.congress.gov/img/member/y000067_20...,"349 Cannon House Office Building, Washington, ...",(202) 225-3915,https://yakym.house.gov,H2IN02295,"YAKYM, Rudy, III",House,0.513,IN,2,22171,Republican


In [42]:
data2 = data[['full_name', 'state_abbrev', 'district_code', 'party']]
data2['party']= data2['party'].str[0]
display_name = [n + ' (' + p + ", "+ s + "-" + str(d) + ')'
                for n, s, d, p in zip(data2['full_name'],
                                 data2['state_abbrev'], 
                                 data2['district_code'],
                                 data2['party'])]

display_name = [x.replace('-0', '') for x in display_name]

display_name

['Robert B. Aderholt (R, AL-4)',
 'Jake Auchincloss (D, MA-4)',
 'Mark E. Amodei (R, NV-2)',
 'Alma S. Adams (D, NC-12)',
 'Pete Aguilar (D, CA-33)',
 'Rick W. Allen (R, GA-12)',
 'Jodey C. Arrington (R, TX-19)',
 'Mark Alford (R, MO-4)',
 'Gabe Amo (D, RI-1)',
 'Yassamin Ansari (D, AZ-3)',
 'Angela D. Alsobrooks (D, MD)',
 'Sanford D. Bishop, Jr. (D, GA-2)',
 'Cliff Bentz (R, OR-2)',
 'Stephanie I. Bice (R, OK-5)',
 'Lauren Boebert (R, CO-4)',
 'Tammy Baldwin (D, WI)',
 'John Boozman (R, AR)',
 'Marsha Blackburn (R, TN)',
 'Gus M. Bilirakis (R, FL-12)',
 'Vern Buchanan (R, FL-16)',
 'John Barrasso (R, WY)',
 'Michael F. Bennet (D, CO)',
 'Richard Blumenthal (D, CT)',
 'Suzanne Bonamici (D, OR-1)',
 'Joyce Beatty (D, OH-3)',
 'Andy Barr (R, KY-6)',
 'Julia Brownley (D, CA-26)',
 'Ami Bera (D, CA-6)',
 'Cory A. Booker (D, NJ)',
 'Brian Babin (R, TX-36)',
 'Donald S. Beyer, Jr. (D, VA-8)',
 'Mike Bost (R, IL-12)',
 'Brendan F. Boyle (D, PA-2)',
 'Don Bacon (R, NE-2)',
 'Jim Banks (R, IN)

In [43]:
b = 'A000055'

myquery =f'''
SELECT *
FROM members
WHERE bioguide_id = '{b}'
'''

member_info = pd.read_sql_query(myquery, con=engine)

member_info = member_info.drop(['bioguide_id', 'image', 'fec_id', 'bioname', 'icpsr'], axis = 1)
ff.create_table(member_info.T.rename({'index': '',0:''},axis = 1).reset_index())

In [44]:

myquery =f'''
    SELECT *
    FROM members
    WHERE bioguide_id = '{b}'
    '''
pd.read_sql_query(myquery, con=engine)['image'][0]

'https://www.congress.gov/img/member/a000055_200.jpg'

In [48]:
myquery = f"""
SELECT c.comparison_member,
    c.agree,
    m.left_right_ideology,
    m.party
FROM members m
INNER JOIN (SELECT vc.comparison_member,
    vc.agree
FROM members m
INNER JOIN vote_compare vc
    ON m.bioname = vc.bioname
WHERE bioguide_id = '{b}') c
    ON m.bioname = c.comparison_member
"""

vote_data = pd.read_sql_query(myquery, con=engine)

In [46]:
import plotly.express as px

In [54]:

fig = px.scatter(vote_data,
                 x = 'left_right_ideology',
                 y = 'agree',
                 hover_name = "comparison_member",
                 color = 'party',
                 color_discrete_map={'Republican': 'red', 'Democrat': 'blue', 'Independent': 'green'})

fig.show()
